# DIC Phase 2
## Prof. Eric Mikida
### Yashika Nihalani, 50425015
### Ashutosh Shailesh Bhawsar, 50416025

In [1]:
import pandas as pd
import numpy as np

import time

import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime



In [7]:
btc_data = pd.read_csv('../data/bitcoin_daily_data_cleaned.csv')

btc_data

,Timestamp,Open,Close,High,Low,Volume_BTC,Volume_Currency,Weighted_Price,Change_percentage,SMA10,SMA20,SMA50,Trade_signal
0,2011-12-31,4.39,4.58,4.58,4.39,23.83,106.33,4.47,NaN,NaN,NaN,NaN,0.0
1,2012-01-01,4.58,5.00,5.00,4.58,7.20,35.26,4.81,9.17,NaN,NaN,NaN,0.0
2,2012-01-02,5.00,5.00,5.00,5.00,19.05,95.24,5.00,0.00,NaN,NaN,NaN,0.0
3,2012-01-03,5.32,5.29,5.32,5.14,11.00,58.10,5.25,5.80,NaN,NaN,NaN,0.0
4,2012-01-04,4.93,5.57,5.57,4.93,11.91,63.12,5.21,5.29,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,2021-03-27,55081.26,55839.42,56686.15,53948.35,1.82,100884.73,55193.36,1.38,55420.714,56204.7735,51692.5584,1.0
3375,2021-03-28,55817.85,55790.92,56573.04,54677.51,1.45,80632.12,55832.96,-0.09,55235.001,56372.9295,52022.7348,1.0
3376,2021-03-29,55790.28,57600.10,58402.68,54892.42,3.73,213754.56,56913.99,3.24,55187.811,56506.4835,52397.5690,1.0
3377,2021-03-30,57623.66,58760.59,59388.66,57011.00,2.36,138231.24,58346.91,2.01,55250.031,56651.8985,52644.4518,1.0


# **Algorithms and their Visualizations**

##### **1) Linear Regression**

##### **2) KNN Classifier**

##### **3) Random Forest Regressor**

##### **4) Long Short Term Memory (LSTM)**

In [ ]:

from keras.layers import LSTM, Dense, Dropout, Activation
from keras.models import Sequential

##### **5) Transformers**